In [12]:
import pandas as pd 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Выкидываем лишние данные (агрегацию по ФО и по России)

In [138]:
vrp = pd.read_excel('vrp16.xls')

In [139]:
federal_districts = [
    "Российская Федерация",
    "    Центральный федеральный округ",
    "    Северо-Западный федеральный округ",
    "            Ненецкий автономный округ (Архангельская область)",
    "            Архангельская область (кроме Ненецкого автономного округа)",
    "    Южный федеральный округ (по 2009 год)",
    "    Южный федеральный округ (с 2010 года)",
    "    Южный федеральный округ (с 29.07.2016)",
    "    Северо-Кавказский федеральный округ",
    "    Приволжский федеральный округ",
    "    Уральский федеральный округ",
    "            Ханты-Мансийский автономный округ - Югра (Тюменская область)",
    "            Ямало-Ненецкий автономный округ (Тюменская область)",
    "            Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)",
    "    Сибирский федеральный округ (с 03.11.2018)",
    "            Таймырский (Долгано-Ненецкий) автономный округ (Красноярский край)",
    "            Эвенкийский автономный округ (Красноярский край)",
    "    Дальневосточный федеральный округ (с 03.11.2018)",
    "            Агинский Бурятский округ (Забайкальский край)",
    "            Корякский округ, входящий в состав Камчатского края",
    "    Крымский федеральный округ", 
    "            Усть-Ордынский Бурятский округ"
]


federal_districts_2 = [
    "Российская Федерация",
    "    Центральный федеральный округ",
    "    Северо-Западный федеральный округ",
    "            Ненецкий автономный округ (Архангельская область)",
    "            Архангельская область (кроме Ненецкого автономного округа)",
    "    Южный федеральный округ (с 29.07.2016)",
    "    Северо-Кавказский федеральный округ",
    "    Приволжский федеральный округ",
    "    Уральский федеральный округ",
    "            Ханты-Мансийский автономный округ - Югра (Тюменская область)",
    "            Ямало-Ненецкий автономный округ (Тюменская область)",
    "            Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)",
    "    Сибирский федеральный округ (с 03.11.2018)",
    "    Дальневосточный федеральный округ (с 03.11.2018)",
    "Сухопутные территории Арктической зоны Российской Федерации"
]


In [140]:
vrp_1 = vrp[~vrp['region'].isin(federal_districts)]
vrp_1 = vrp_1[~vrp_1['region'].isin(federal_districts_2)]

vrp_1['region'] = vrp_1['region'].str.strip()
len(vrp_1['region'].unique())
vrp_1['region'].unique()

82

array(['Белгородская область', 'Брянская область', 'Владимирская область',
       'Воронежская область', 'Ивановская область', 'Калужская область',
       'Костромская область', 'Курская область', 'Липецкая область',
       'Московская область', 'Орловская область', 'Рязанская область',
       'Смоленская область', 'Тамбовская область', 'Тверская область',
       'Тульская область', 'Ярославская область',
       'Город Москва столица Российской Федерации город федерального значения',
       'Республика Карелия', 'Республика Коми', 'Архангельская область',
       'Вологодская область', 'Калининградская область',
       'Ленинградская область', 'Мурманская область',
       'Новгородская область', 'Псковская область',
       'Город Санкт-Петербург город федерального значения',
       'Республика Адыгея (Адыгея)', 'Республика Калмыкия',
       'Республика Крым', 'Краснодарский край', 'Астраханская область',
       'Волгоградская область', 'Ростовская область',
       'Город федерального зн

In [ ]:
# vrp15 = vrp_1.copy()
# year_columns = ['2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
# vrp15[year_columns] = vrp15[year_columns] * 1000
# vrp15 = vrp15.drop(columns=['amt'])
# vrp15.to_excel('vrp15_1.xlsx', index=False)
# vrp_1.to_excel('vrp16_1.xlsx', index=False)

## Объединение разделов

In [144]:
df = pd.read_excel('keys_okved.xlsx')
data = list(df[['okved2_letter', 'okved2007_letter']].itertuples(index=False, name=None))

from collections import defaultdict

def build_left_right_sets(data):
    """
    data: список (left_letter, right_letter).
    Возвращает dict: left_letter -> set(right_letters).
    """
    mapping = defaultdict(set)
    for left, right in data:
        mapping[left].add(right)
    return dict(mapping)

def dict_to_list_of_pairs(mapping):
    """
    Принимает dict: letter -> set_of_right_letters
    Возвращает список кортежей: ( {left_letter}, set_of_right_letters ).
    """
    rows = []
    for left, right_set in mapping.items():
        rows.append( ( {left}, right_set ) )
    return rows

def merge_rows_by_right_intersection(rows):
    """
    Принимает список кортежей (left_set, right_set).
    Пока есть пары с непустым пересечением right_set, сливает их.
    Возвращает итоговый список (left_set, right_set).
    """
    merged = True
    while merged:
        merged = False
        n = len(rows)
        if n <= 1:
            break
        
        i = 0
        while i < n-1:
            j = i + 1
            while j < n:
                left_i, right_i = rows[i]
                left_j, right_j = rows[j]
                
                # Проверяем пересечение правых множеств
                if right_i.intersection(right_j):
                    # Нашли непустое пересечение -> сливаем
                    new_left = left_i.union(left_j)
                    new_right = right_i.union(right_j)
                    
                    # Сформируем новый список без i и j
                    new_rows = []
                    for k in range(n):
                        if k not in (i, j):
                            new_rows.append(rows[k])
                    
                    # Добавляем объединённую строку
                    new_rows.append( (new_left, new_right) )
                    
                    # Заменяем rows
                    rows = new_rows
                    merged = True
                    break  # выход из inner while
                j += 1
            
            if merged:
                break  # заново while i
            i += 1
    
    return rows


    
mapping = build_left_right_sets(data)
rows = dict_to_list_of_pairs(mapping)
merged_rows = merge_rows_by_right_intersection(rows)

print("Итоговые строки (после объединения по правым множествам):")
for (lefts, rights) in merged_rows:
    print(f"Left: {sorted(lefts)},  Right: {sorted(rights)}")


Итоговые строки (после объединения по правым множествам):
Left: ['A'],  Right: ['A', 'B']
Left: ['B'],  Right: ['C']
Left: ['F'],  Right: ['F']
Left: ['G'],  Right: ['G']
Left: ['I'],  Right: ['H']
Left: ['K'],  Right: ['J']
Left: ['O'],  Right: ['L']
Left: ['P'],  Right: ['M']
Left: ['T'],  Right: ['P']
Left: ['U'],  Right: ['Q']
Left: ['C', 'D', 'E', 'H', 'J', 'L', 'M', 'N', 'Q', 'R', 'S'],  Right: ['D', 'E', 'I', 'K', 'N', 'O']


In [196]:
vrp15 = pd.read_excel('vrp15_1.xlsx')
vrp16 = pd.read_excel('vrp16_1.xlsx')
vrp15 = vrp15.dropna(subset='section')
vrp16 = vrp16.dropna(subset='section')

# vrp15.groupby('section', as_index=False)['region'].nunique()
# vrp16.groupby('section', as_index=False)['region'].nunique()

In [197]:
year_columns = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
vrp16[year_columns] = vrp16[year_columns] * 1000

In [198]:
def transform_excel_data(df) -> pd.DataFrame:
    year_cols = []
    for col in df.columns:
        try:
            _ = int(col)
            year_cols.append(col)
        except ValueError:
            pass
    year_cols = sorted(year_cols, key=int)
    df_long = df.melt(
        id_vars=['section', 'section_desc', 'region'],
        value_vars=year_cols,
        var_name='year',
        value_name='value'
    )
    df_pivot = df_long.pivot(
        index=['region', 'year'],
        columns='section',
        values='value'
    ).reset_index()
    return df_pivot

vrp15_melted = transform_excel_data(vrp15)
vrp16_melted = transform_excel_data(vrp16)

In [199]:
# -------------------------------------------------------------------
#  1) ({'A'}, {'A','B'}) -> df_a_ab
# -------------------------------------------------------------------
df_left_a_ab = vrp16_melted[['region','year']].copy()
df_left_a_ab['A_AB'] = vrp16_melted['A'].fillna(0)

df_right_a_ab = vrp15_melted[['region','year']].copy()
df_right_a_ab['A_AB'] = vrp15_melted[['A','B']].fillna(0).sum(axis=1)

df_a_ab = pd.concat([df_left_a_ab, df_right_a_ab], ignore_index=True)

# -------------------------------------------------------------------
#  2) ({'B'}, {'C'}) -> df_b_c
# -------------------------------------------------------------------
df_left_b_c = vrp16_melted[['region','year']].copy()
df_left_b_c['B_C'] = vrp16_melted['B'].fillna(0)

df_right_b_c = vrp15_melted[['region','year']].copy()
df_right_b_c['B_C'] = vrp15_melted['C'].fillna(0)

df_b_c = pd.concat([df_left_b_c, df_right_b_c], ignore_index=True)

# -------------------------------------------------------------------
#  3) ({'F'}, {'F'}) -> df_f_f
# -------------------------------------------------------------------
df_left_f_f = vrp16_melted[['region','year']].copy()
df_left_f_f['F_F'] = vrp16_melted['F'].fillna(0)

df_right_f_f = vrp15_melted[['region','year']].copy()
df_right_f_f['F_F'] = vrp15_melted['F'].fillna(0)

df_f_f = pd.concat([df_left_f_f, df_right_f_f], ignore_index=True)

# -------------------------------------------------------------------
#  4) ({'G'}, {'G'}) -> df_g_g
# -------------------------------------------------------------------
df_left_g_g = vrp16_melted[['region','year']].copy()
df_left_g_g['G_G'] = vrp16_melted['G'].fillna(0)

df_right_g_g = vrp15_melted[['region','year']].copy()
df_right_g_g['G_G'] = vrp15_melted['G'].fillna(0)

df_g_g = pd.concat([df_left_g_g, df_right_g_g], ignore_index=True)

# -------------------------------------------------------------------
#  5) ({'I'}, {'H'}) -> df_i_h
# -------------------------------------------------------------------
df_left_i_h = vrp16_melted[['region','year']].copy()
df_left_i_h['I_H'] = vrp16_melted['I'].fillna(0)

df_right_i_h = vrp15_melted[['region','year']].copy()
df_right_i_h['I_H'] = vrp15_melted['H'].fillna(0)

df_i_h = pd.concat([df_left_i_h, df_right_i_h], ignore_index=True)

# -------------------------------------------------------------------
#  6) ({'K'}, {'J'}) -> df_k_j
# -------------------------------------------------------------------
df_left_k_j = vrp16_melted[['region','year']].copy()
df_left_k_j['K_J'] = vrp16_melted['K'].fillna(0)

df_right_k_j = vrp15_melted[['region','year']].copy()
df_right_k_j['K_J'] = vrp15_melted['J'].fillna(0)

df_k_j = pd.concat([df_left_k_j, df_right_k_j], ignore_index=True)

# -------------------------------------------------------------------
#  7) ({'O'}, {'L'}) -> df_o_l
# -------------------------------------------------------------------
df_left_o_l = vrp16_melted[['region','year']].copy()
df_left_o_l['O_L'] = vrp16_melted['O'].fillna(0)

df_right_o_l = vrp15_melted[['region','year']].copy()
df_right_o_l['O_L'] = vrp15_melted['L'].fillna(0)

df_o_l = pd.concat([df_left_o_l, df_right_o_l], ignore_index=True)

# -------------------------------------------------------------------
#  8) ({'P'}, {'M'}) -> df_p_m
# -------------------------------------------------------------------
df_left_p_m = vrp16_melted[['region','year']].copy()
df_left_p_m['P_M'] = vrp16_melted['P'].fillna(0)

df_right_p_m = vrp15_melted[['region','year']].copy()
df_right_p_m['P_M'] = vrp15_melted['M'].fillna(0)

df_p_m = pd.concat([df_left_p_m, df_right_p_m], ignore_index=True)

# -------------------------------------------------------------------
#  9) ({'T'}, {'P'}) -> df_t_p
# -------------------------------------------------------------------
df_left_t_p = vrp16_melted[['region','year']].copy()
df_left_t_p['T_P'] = vrp16_melted['T'].fillna(0)

df_right_t_p = vrp15_melted[['region','year']].copy()
df_right_t_p['T_P'] = vrp15_melted['P'].fillna(0)

df_t_p = pd.concat([df_left_t_p, df_right_t_p], ignore_index=True)

# -------------------------------------------------------------------
# 10) ({'U'}, {'Q'}) -> df_u_q
# -------------------------------------------------------------------
# df_left_u_q = vrp16_melted[['region','year']].copy()
# df_left_u_q['U_Q'] = vrp16_melted['U'].fillna(0)

# df_right_u_q = vrp15_melted[['region','year']].copy()
# df_right_u_q['U_Q'] = vrp15_melted['Q'].fillna(0)

# df_u_q = pd.concat([df_left_u_q, df_right_u_q], ignore_index=True)

# -------------------------------------------------------------------
# 11) ({'C','D','E','H','J','L','M','N','Q','R','S'},
#      {'D','E','I','K','N','O'}) -> df_big
# -------------------------------------------------------------------
df_left_big = vrp16_melted[['region','year']].copy()
df_left_big['CDEHJLMNQRS_DEIKNO'] = (
    vrp16_melted[['C','D','E','H','J','L','M','N','Q','R','S']].fillna(0).sum(axis=1)
)

df_right_big = vrp15_melted[['region','year']].copy()
df_right_big['CDEHJLMNQRS_DEIKNO'] = (
    vrp15_melted[['D','E','I','K','N','O']].fillna(0).sum(axis=1)
)

df_big = pd.concat([df_left_big, df_right_big], ignore_index=True)


# ===================================================================
#             ОБЪЕДИНЯЕМ (MERGE) ВСЕ ТАБЛИЦЫ В ОДНУ
# ===================================================================

df_final = df_a_ab.merge(df_b_c, on=['region','year'], how='outer')
df_final = df_final.merge(df_f_f, on=['region','year'], how='outer')
df_final = df_final.merge(df_g_g, on=['region','year'], how='outer')
df_final = df_final.merge(df_i_h, on=['region','year'], how='outer')
df_final = df_final.merge(df_k_j, on=['region','year'], how='outer')
df_final = df_final.merge(df_o_l, on=['region','year'], how='outer')
df_final = df_final.merge(df_p_m, on=['region','year'], how='outer')
df_final = df_final.merge(df_t_p, on=['region','year'], how='outer')
# df_final = df_final.merge(df_u_q, on=['region','year'], how='outer')
df_final = df_final.merge(df_big, on=['region','year'], how='outer')
df_final.sort_values(by = ['region', 'year'])

section,region,year,A_AB,B_C,F_F,G_G,I_H,K_J,O_L,P_M,T_P,CDEHJLMNQRS_DEIKNO
574,Алтайский край,2004,2.672922e+10,1.676973e+08,4.567839e+09,2.203586e+10,6.777187e+08,1.171503e+08,5.504922e+09,5.907413e+09,0.0,4.913266e+10
575,Алтайский край,2005,2.286518e+10,2.544510e+08,6.424661e+09,2.947991e+10,1.046310e+09,1.364050e+08,6.767331e+09,7.039828e+09,0.0,6.167237e+10
576,Алтайский край,2006,2.829541e+10,7.326142e+08,8.333709e+09,3.640132e+10,1.409747e+09,2.873180e+08,1.242688e+10,7.945219e+09,0.0,7.797830e+10
577,Алтайский край,2007,3.523437e+10,1.158740e+09,9.280378e+09,5.309161e+10,1.778273e+09,3.232340e+08,1.600080e+10,9.248980e+09,0.0,9.744702e+10
578,Алтайский край,2008,4.027391e+10,1.751855e+09,1.127410e+10,5.534055e+10,2.305733e+09,3.703230e+08,1.978199e+10,1.132118e+10,0.0,1.169235e+11
...,...,...,...,...,...,...,...,...,...,...,...,...
569,Ярославская область,2018,1.757414e+10,7.322401e+08,2.896180e+10,8.938170e+10,5.041778e+09,1.407296e+09,2.866025e+10,1.881267e+10,0.0,3.930334e+11
570,Ярославская область,2019,1.992335e+10,9.108024e+08,3.026452e+10,9.029811e+10,7.259083e+09,1.225699e+09,2.942348e+10,1.987774e+10,0.0,4.099680e+11
571,Ярославская область,2020,2.015658e+10,8.092309e+08,2.619824e+10,8.625735e+10,5.064922e+09,1.935217e+09,3.140720e+10,2.139781e+10,0.0,4.185219e+11
572,Ярославская область,2021,2.826518e+10,1.165085e+09,3.315411e+10,1.014020e+11,7.356248e+09,1.858613e+09,3.533247e+10,2.813779e+10,0.0,4.583439e+11


In [200]:
df_final['vrp'] = df_final[['A_AB', 'B_C', 'F_F', 'G_G', 'I_H', 'K_J', 'O_L', 'P_M', 'T_P', 'CDEHJLMNQRS_DEIKNO']].sum(axis=1)
vrp = df_final[['region', 'year', 'vrp']]
vrp

section,region,year,vrp
0,Алтайский край,2016,5.324015e+11
1,Алтайский край,2017,5.453030e+11
2,Алтайский край,2018,5.797405e+11
3,Алтайский край,2019,6.281461e+11
4,Алтайский край,2020,6.641292e+11
...,...,...,...
1553,Ярославская область,2011,2.869675e+11
1554,Ярославская область,2012,3.272796e+11
1555,Ярославская область,2013,3.628618e+11
1556,Ярославская область,2014,3.914628e+11


## Реальный ВРП

In [218]:
ifo = pd.read_excel('ifo.xlsx')
ifo = ifo.drop(columns = ['region2'])
ifo['section'] = 'ifo'

def transform_excel_data(df) -> pd.DataFrame:
    year_cols = []
    for col in df.columns:
        try:
            _ = int(col)
            year_cols.append(col)
        except ValueError:
            pass
    year_cols = sorted(year_cols, key=int)
    df_long = df.melt(
        id_vars=['section', 'region'],
        value_vars=year_cols,
        var_name='year',
        value_name='value'
    )
    df_pivot = df_long.pivot(
        index=['region', 'year'],
        columns='section',
        values='value'
    ).reset_index()
    return df_pivot

ifo_melted = transform_excel_data(ifo)
ifo_melted['ifo'] = ifo_melted['ifo']/100

In [219]:
def compute_cumulative_index(df, base_year=2016):
    # Убедимся, что год – числовое значение, и отсортируем данные
    df = df.copy()
    df['year'] = df['year'].astype(int)
    df = df.sort_values(['region', 'year'])
    
    def process_region(gr):
        gr = gr.sort_values('year').copy()
        # Разделим данные на "до базы" (включая base_year) и "после базы"
        df_pre = gr[gr['year'] <= base_year].copy()
        df_post = gr[gr['year'] > base_year].copy()
        
        # Для годов до 2016: ifo_cum_index = произведение (ifo/100) для всех годов от t+1 до 2016
        if not df_pre.empty:
            # Перевернём порядок, чтобы сделать обратное кумулятивное произведение
            df_pre_rev = df_pre.iloc[::-1].copy()
            df_pre_rev['cumprod'] = (df_pre_rev['ifo']).cumprod()
            # Сдвигаем на одну строку: для каждой строки берем произведение коэффициентов следующих годов
            df_pre_rev['ifo_cum_index'] = df_pre_rev['cumprod'].shift(1)
            # Для самого последнего года (базового, 2016) задаём ifo_cum_index = 1
            df_pre_rev.iloc[0, df_pre_rev.columns.get_loc('ifo_cum_index')] = 1.0
            df_pre = df_pre_rev.iloc[::-1].copy()
        
        # Для годов после 2016:
        if not df_post.empty:
            df_post = df_post.sort_values('year').reset_index(drop=True).copy()
            # Для первого года после базы (2017)
            df_post.loc[0, 'ifo_cum_index'] = 1 / df_post.loc[0, 'ifo']
            # Для остальных: перемножаем коэффициенты ifo/100
            if len(df_post) > 1:
                factors = (df_post['ifo']).iloc[1:]
                cumprod = factors.cumprod()
                df_post.loc[df_post.index[1:], 'ifo_cum_index'] = (1 / df_post.loc[0, 'ifo']) * cumprod
        
        return pd.concat([df_pre, df_post]).sort_values('year')
    
    result = df.groupby('region', group_keys=False).apply(process_region)
    return result

ifo_melted = compute_cumulative_index(ifo_melted, base_year=2016)
ifo_melted = ifo_melted[['region', 'year', 'ifo_cum_index']] #'region', 'year', 'ifo', 'cum_index'
ifo_melted

section,region,year,ifo_cum_index
0,Алтайский край,1998,2.017821
1,Алтайский край,1999,1.962861
2,Алтайский край,2000,1.757262
3,Алтайский край,2001,1.640767
4,Алтайский край,2002,1.580701
...,...,...,...
1,Ярославская область,2018,1.008798
2,Ярославская область,2019,1.009806
3,Ярославская область,2020,1.006777
4,Ярославская область,2021,1.059129


In [220]:
vrp['year'] = vrp['year'].astype(int)

real_vrp = vrp.merge(ifo_melted, on = ['region', 'year'], how = 'inner')
real_vrp['real_vrp'] = real_vrp['vrp'] * real_vrp['ifo_cum_index']
real_vrp = real_vrp.drop(columns=['ifo_cum_index'])
real_vrp.sort_values(by = ['region', 'year'])

C:\Users\lizai\AppData\Local\Temp\ipykernel_15140\406097341.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vrp['year'] = vrp['year'].astype(int)


section,region,year,vrp,real_vrp
574,Алтайский край,2004,1.148405e+11,1.572295e+11
575,Алтайский край,2005,1.356864e+11,1.828445e+11
576,Алтайский край,2006,1.738105e+11,2.125396e+11
577,Алтайский край,2007,2.235634e+11,2.496608e+11
578,Алтайский край,2008,2.593431e+11,2.803652e+11
...,...,...,...,...
569,Ярославская область,2018,5.836053e+11,5.887396e+11
570,Ярославская область,2019,6.091508e+11,6.151244e+11
571,Ярославская область,2020,6.117485e+11,6.158943e+11
572,Ярославская область,2021,6.950154e+11,7.361112e+11


In [221]:
pop = pd.read_excel('population.xlsx')
pop['region'] = pop['region'].str.strip()
# len(pop['region'].unique())
# pop['region'].unique()
pop['section'] = 'population'

def transform_excel_data(df) -> pd.DataFrame:
    year_cols = []
    for col in df.columns:
        try:
            _ = int(col)
            year_cols.append(col)
        except ValueError:
            pass
    year_cols = sorted(year_cols, key=int)
    df_long = df.melt(
        id_vars=['section', 'region'],
        value_vars=year_cols,
        var_name='year',
        value_name='value'
    )
    df_pivot = df_long.pivot(
        index=['region', 'year'],
        columns='section',
        values='value'
    ).reset_index()
    return df_pivot

pop_melted = transform_excel_data(pop)

In [222]:
real_vrp_pop = real_vrp.merge(pop_melted, on = ['region', 'year'], how = 'inner')
real_vrp_pop['real_vrp_pop'] = real_vrp_pop['real_vrp'] / real_vrp_pop['population']
real_vrp_pop.sort_values(by = ['region', 'year'])

section,region,year,vrp,real_vrp,population,real_vrp_pop
574,Алтайский край,2004,1.148405e+11,1.572295e+11,2555708.0,61520.932808
575,Алтайский край,2005,1.356864e+11,1.828445e+11,2521470.0,72515.037198
576,Алтайский край,2006,1.738105e+11,2.125396e+11,2488267.0,85416.724638
577,Алтайский край,2007,2.235634e+11,2.496608e+11,2463239.0,101354.696264
578,Алтайский край,2008,2.593431e+11,2.803652e+11,2446182.0,114613.375681
...,...,...,...,...,...,...
569,Ярославская область,2018,5.836053e+11,5.887396e+11,1247850.0,471803.193176
570,Ярославская область,2019,6.091508e+11,6.151244e+11,1239717.0,496181.296747
571,Ярославская область,2020,6.117485e+11,6.158943e+11,1228638.0,501282.158710
572,Ярославская область,2021,6.950154e+11,7.361112e+11,1213650.0,606526.798141


In [223]:
real_vrp_pop = real_vrp_pop.rename(columns={
    'region': 'i',
    'year': 't',
    'real_vrp_pop':'vrp_per_capita'
})

real_vrp_pop = real_vrp_pop.sort_values(by = ['i', 't'])

In [225]:
real_vrp_pop.to_excel('data1302.xlsx', index=False)
real_vrp_pop.to_csv('data1302.csv', index=False)